In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from keras import Input
from keras.layers import BatchNormalization, Dropout, Dense, TimeDistributed,Dropout, Activation,Lambda
import keras.backend as K
from keras.models import Sequential, Model
from  keras.layers import concatenate
import numpy as np
import pandas as pd

In [0]:
data = pd.read_csv('/content/drive/My Drive/merge_all_flatten.csv')    
train_label = pd.read_csv('/content/drive/My Drive/train_profit.csv')
train_label = train_label.sort_values('acc_id')

In [30]:
# model 1 
input_1 = Input(shape = (400,), name='input_1')
x = Dense(800,activation='relu')(input_1)
x = Dense(1000, activation='relu')(x)
x = Dense(2000, activation='relu')(x)
x = Dense(2000, activation='relu')(x)
x = Dense(1000, activation='relu')(x)
x = Dense(500, activation='relu')(x)
x = Dense(1)(x)
#x = Activation('softmax',name="survival")(x)
x = Activation('relu',name="survival")(x)

# model 2
y = Dense(400,activation='relu')(input_1)
y = Dense(800, activation='relu')(y)
y = Dense(100, activation='relu')(y)
y = Dense(2000, activation='relu')(y)
y = Dense(2000, activation='relu')(y)
y = Dense(400, activation='relu')(y)
y = Dense(1)(y)
y = Activation('relu',name="amount")(y)

# loss
#def loss(x,y):
#    return K.min(y_true-y_pred)
    
# Concat x, y
#k = Lambda(lambda x: K.argmax(x, axis=-1))(x)
z = concatenate([x, y])
z = Dense(500, activation='relu')(z)
z = Dense(1000, activation='relu')(z)
z = Dense(500, activation='relu')(z)
z = Dense(1)(z)
z = Activation('relu', name="profit")(z)


# model
model = Model(inputs=input_1, outputs=[x,y,z])

losses = {
    #"survival": "sparse_categorical_crossentropy",
    "survival": "mse",
    "amount": "mse",
    "profit": "mse"}

model.compile(optimizer='rmsprop',
              loss=losses,
              metrics=['mse'])    

model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
dense_211 (Dense)               (None, 800)          320800      input_1[0][0]                    
__________________________________________________________________________________________________
dense_218 (Dense)               (None, 400)          160400      input_1[0][0]                    
__________________________________________________________________________________________________
dense_212 (Dense)               (None, 1000)         801000      dense_211[0][0]                  
____________________________________________________________________________________________

In [31]:
model.fit(data[data.columns[1:]], {"survival": train_label['survival_time'], "amount": train_label['amount_spent'],
                   "profit": train_label['E_profit']}, epochs=100, batch_size=128)

Epoch 1/100
40000/40000 [==============================] - 13s 317us/step - loss: 3101.0709 - survival_loss: 2642.4984 - amount_loss: 0.5373 - profit_loss: 458.0352 - survival_mean_squared_error: 2642.4984 - amount_mean_squared_error: 0.5373 - profit_mean_squared_error: 458.0352
Epoch 2/100
40000/40000 [==============================] - 11s 277us/step - loss: 3088.9050 - survival_loss: 2630.3913 - amount_loss: 0.5373 - profit_loss: 457.9765 - survival_mean_squared_error: 2630.3913 - amount_mean_squared_error: 0.5373 - profit_mean_squared_error: 457.9765
Epoch 3/100
40000/40000 [==============================] - 11s 277us/step - loss: 3088.8256 - survival_loss: 2630.3913 - amount_loss: 0.5373 - profit_loss: 457.8970 - survival_mean_squared_error: 2630.3913 - amount_mean_squared_error: 0.5373 - profit_mean_squared_error: 457.8970
Epoch 4/100
40000/40000 [==============================] - 11s 275us/step - loss: 3088.8480 - survival_loss: 2630.3913 - amount_loss: 0.5373 - profit_loss: 457.

In [0]:
k =model.predict(data[data.columns[1:]])